# Fine Tuning

- 기존 모델의 학습 시간이 GPU를 사용함에도 매우 오래걸려 라이브러리를 이용해 재구성했습니다.

- 데이터는 CoLa data set을 사용했고, 이에 대한 적절한 작업을 수행하기 위해, Classification을 수행하는 XLNet을 가져왔습니다.

# 필요한 라이브러리 import

In [46]:
!pip install transformers==3.3.0
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 20.9 MB/s 
     |████████████████████████████████| 880 kB 49.4 MB/s 
     |████████████████████████████████| 3.0 MB 57.8 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=6190f3e64938e8312ff2700abdf8b544501bca15b6877dde186c3e21e9b8b27b
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.2
    Uninstalling tokenizers-0.13.2:
      Successfully uninstalled tokenizers-0.13.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.25.1
    Uninstalling transformers-4.25.1:
      Successfully uninstalled transformers-4.25.1


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [47]:
from transformers import *
import torch
from transformers import XLNetForSequenceClassification
from transformers import XLNetTokenizer, XLNetModel
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras_preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset,DataLoader,RandomSampler,SequentialSampler
import pandas as pd

# 데이터 전처리

In [36]:
!unzip -qq "/content/cola_public_1.1.zip" -d "/content/cola_public_1.1"

replace /content/cola_public_1.1/cola_public/README? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/cola_public_1.1/cola_public/tokenized/in_domain_dev.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/cola_public_1.1/cola_public/tokenized/in_domain_train.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/cola_public_1.1/cola_public/tokenized/out_of_domain_dev.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/cola_public_1.1/cola_public/raw/in_domain_dev.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/cola_public_1.1/cola_public/raw/in_domain_train.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/cola_public_1.1/cola_public/raw/out_of_domain_dev.tsv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y


In [48]:
df = pd.read_csv("/content/cola_public_1.1/cola_public/raw/in_domain_train.tsv", delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])
df.head(3)

,sentence_source,label,label_notes,sentence
0,gj04,1,NaN,"Our friends won't buy this analysis, let alone..."
1,gj04,1,NaN,One more pseudo generalization and I'm giving up.
2,gj04,1,NaN,One more pseudo generalization or I'm giving up.


In [49]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [50]:
device

device(type='cuda')

In [51]:
df=df[['sentence','label']]
df.head(3)

,sentence,label
0,"Our friends won't buy this analysis, let alone...",1
1,One more pseudo generalization and I'm giving up.,1
2,One more pseudo generalization or I'm giving up.,1


In [52]:
sentences  = []
for sentence in df['sentence']:
  sentence = sentence+"[SEP] [CLS]"
  sentences.append(sentence)

In [53]:
# questions_2  = []
# for question in df['question2']:
#   question = question+"[SEP] [CLS]"
#   questions_2.append(question)

In [54]:
# questions = []
# questions = questions_1 + questions_2

In [56]:
tokenizer = XLNetTokenizer.from_pretrained('xlnet-base-cased')
model = XLNetModel.from_pretrained('xlnet-base-cased')
tokenized_text = [tokenizer.tokenize(sent) for sent in sentences]

TypeError: ignored

In [ ]:
ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_text]

In [ ]:
labels = df['label'].values

In [ ]:
max_ = len(ids[0])
for i in ids:
  max_=max(max_,len(i))
print(max_)
MAX_LEN = max_

54


In [ ]:
input_ids2 = pad_sequences(ids,maxlen=MAX_LEN,dtype="long",truncating="post",padding="post")

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(input_ids2,labels,test_size=0.15)

In [ ]:
Xtrain = torch.tensor(xtrain)
Ytrain = torch.tensor(ytrain)
Xtest = torch.tensor(xtest)
Ytest = torch.tensor(ytest)

In [ ]:
batch_size = 4
no_train = 0
epochs = 10

In [ ]:
train_data = TensorDataset(Xtrain,Ytrain)
test_data = TensorDataset(Xtest,Ytest)
loader = DataLoader(train_data,batch_size=batch_size)
test_loader = DataLoader(test_data,batch_size=batch_size)

In [ ]:
def flat_accuracy(preds,labels):  # A function to predict Accuracy
  correct=0
  for i in range(0,len(labels)):
    if(preds[i]==labels[i]):
      correct+=1
  return (correct/len(labels))*100
model = XLNetForSequenceClassification.from_pretrained("xlnet-base-cased",num_labels=2) # xlnet-base(large)-cased
torch.cuda.empty_cache()
model.cuda()
optimizer = AdamW(model.parameters(),lr=2e-5) # initial learning rate 0.00002

loading configuration file config.json from cache at C:\Users\woduc/.cache\huggingface\hub\models--xlnet-base-cased\snapshots\593a21e8b79948a7f952811aa44f37d76e23d586\config.json
Model config XLNetConfig {
  "architectures": [
    "XLNetLMHeadModel"
  ],
  "attn_type": "bi",
  "bi_data": false,
  "bos_token_id": 1,
  "clamp_len": -1,
  "d_head": 64,
  "d_inner": 3072,
  "d_model": 768,
  "dropout": 0.1,
  "end_n_top": 5,
  "eos_token_id": 2,
  "ff_activation": "gelu",
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-12,
  "mem_len": null,
  "model_type": "xlnet",
  "n_head": 12,
  "n_layer": 12,
  "pad_token_id": 5,
  "reuse_len": null,
  "same_length": false,
  "start_n_top": 5,
  "summary_activation": "tanh",
  "summary_last_dropout": 0.1,
  "summary_type": "last",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 250
    }
  },
  "transformers_version": "4.24.0",
  "untie_r": true,
  "use_mems_eval": true

RuntimeError: CUDA out of memory. Tried to allocate 94.00 MiB (GPU 0; 4.00 GiB total capacity; 3.37 GiB already allocated; 0 bytes free; 3.46 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
import torch.nn as nn
criterion = nn.BCELoss()

# 모델 학습

- 논문에서는 시그모이드 함수를 loss로 정의했다고 나와있지만, 이를 cross entropy를 사용해보니 CoLa데이터셋에 한하여 더 좋은 성능을 보이는 것을 확인할 수 있습니다. 물론, 시간이 지나며 혹은 라이브러리에 들어간 함수의 코드가 달라졌을 가능성을 감안해야 할 것입니다.

In [ ]:
from tqdm import tqdm
for epoch in range(epochs):
  print("-----------------------------------------------------")
  print("Epoch:",epoch+1)
  model.train()
  loss1 = []
  train_loss = []
  l = []
  for inputs,labels1 in tqdm(loader) :
    inputs.to(device)
    labels1.to(device)
    optimizer.zero_grad()
    outputs = model(inputs.to(device))
    labels2 = labels1.to(device)
    loss = criterion(outputs,labels2).to(device)
    [train_loss.append(p.item()) for p in torch.argmax(outputs[0],axis=1).flatten() ]#our predicted 
    [l.append(z.item()) for z in labels1]# real labels
    loss.backward()
    optimizer.step()
    loss1.append(loss.item())
    no_train += inputs.size(0)
  print("Loss :",loss.item())
  print("Accuracy :",flat_accuracy(train_loss,l))
  print("-----------------------------------------------------")

-----------------------------------------------------
Epoch: 1


  0%|          | 0/29 [00:07<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 162.00 MiB (GPU 0; 4.00 GiB total capacity; 3.19 GiB already allocated; 0 bytes free; 3.35 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF